# Training Process

In [ ]:
import pandas as pd
import numpy as np
import spacy
from joblib import dump, load
from sklearn.neural_network import MLPClassifier
from preprocess_data import raw_to_json, unzip_entities, split_entities, tags_patterns_mix, get_responses, remove_fallback

#### Data

In [ ]:
df_train = pd.read_csv('data/Training - Training.csv')
df_test = pd.read_csv('data/Training - Test.csv')
intents = raw_to_json(df_train)
parent_tags = ['navigate', 'find', 'action']
data = unzip_entities(intents, parent_tags)
responses = get_responses(data)
data = remove_fallback(data)
tags_patterns = tags_patterns_mix(data)

In [ ]:
nlp = spacy.load("en_core_web_sm")
def lemmatizer(text):
    doc = nlp(text)
    return [d.lemma_ for d in doc]

def train_pipeline(tags_patterns):
    
    word_list = []
    tags = []
    word_tag_data = []
    
    for i, row in tags_patterns.iterrows():
        tag = row['tag']
        pattern = row['pattern']
        tags.append(tag)
        word = lemmatizer(pattern)
        word_list.extend(word)
        word_tag_data.append((word, tag))
        
    return tags, word_list, word_tag_data

def prepare_training_data(word_tag_data, word_list, tags):
    X = []
    y = []
    
    for (pattern, tag) in word_tag_data:
        bog = bag_of_words(pattern, word_list)
        X.append(bog)
        label = tags.index(tag)
        y.append(label)

    return np.array(X), np.array(y)

def bag_of_words(tokenized_sentence, words):

    bog = np.zeros(len(words), dtype=np.float32)
    for idx, word in enumerate(words):
        if word in tokenized_sentence:
            bog[idx] = 1
    return bog

def pipe_new_input(text):
    if text == '': #otherwise error
        text = ' '
    text = lemmatizer(text)
    bog = bag_of_words(text, word_list)
    x = bog.reshape(1, bog.shape[0])
    return x
    
def predict(text, model):
    x = pipe_new_input(text)
    result = model.predict_proba(x)
    return result

def get_tag_from_prediction(result, tags, threshold, fallback = 'fallback'):
    
    max_proba = np.max(result)
    
    if max_proba < threshold:
        return fallback
    
    predicted_tag = tags[np.argmax(result)]
    
    return predicted_tag
    
def test(df, model, tags, threshold = 0.4, col_name = 'Sentence', col_tag = 'Tag'):
    predicted_tags = []
    matches = []
    probas = []
    predicted_tags_if_not_fallback = []
    
    for i, row in df.iterrows():
        sentence = row[col_name]
        tag = row[col_tag]
        
        result = predict(sentence, model)
        max_proba = np.max(result)
        
        predicted_tag = get_tag_from_prediction(result, tags, threshold)
        
        predicted_tag_except_fallback = get_tag_from_prediction(result, tags, 0.0)
        
        predicted_tags_if_not_fallback.append(predicted_tag_except_fallback)
        probas.append(max_proba)
        predicted_tags.append(predicted_tag)
        matches.append(predicted_tag == tag)
        
    df = df.assign(predicted_tags = predicted_tags, matches = matches, probas = probas, predicted_tags_if_not_fallback = predicted_tags_if_not_fallback)
    
    acc = df['matches'].sum() / len(df)
    output = f'Accuracy: {acc}'
    
    return df, output

def get_random_response_from_tag(tag, responses):
    return np.random.choice(responses[tag])

### Data preparation

In [ ]:
tags, word_list, word_tag_data = train_pipeline(tags_patterns)
IGNORE = ['?', '!', '.', ',']
word_list = [word for word in word_list if word not in IGNORE]
word_list = sorted(set(word_list))
tags = sorted(set(tags))
X, y = prepare_training_data(word_tag_data, word_list, tags)

### Model training & testing

In [ ]:
clf = MLPClassifier(random_state=1, activation = 'logistic', max_iter=50000, hidden_layer_sizes = (16)).fit(X, y)

### Testing
- calculates the accuracy of the model on test data
- plots the dataframe with all misclassified data

In [ ]:
df_test_result, acc = test(df_test, clf, tags, col_name = 'Sentence', col_tag = 'Tag', threshold = 0.25)
print(acc)

Accuracy: 0.9333333333333333

In [ ]:
df_test_result[(df_test_result['matches'] == False)]

|    | Sentence                                 | Tag                  | Tag_parent   | predicted_tags       | matches   |   probas | predicted_tags_if_not_fallback   |
|---:|:-----------------------------------------|:---------------------|:-------------|:---------------------|:----------|---------:|:---------------------------------|
| 17 | Expose the page where I can make a post. | navigate_create_post | navigate     | find_create_trade    | False     | 0.945566 | find_create_trade                |
| 19 | I would like to create a new post.       | action_create_post   | action       | navigate_create_post | False     | 0.736224 | navigate_create_post             |

### Saving the model & data

In [ ]:
dump(clf, 'utils/classifier.model');
dump(responses, 'utils/responses.data');
dump(tags, 'utils/tags.data');
dump(word_list, 'utils/word_list.data');

### Testing the model with own input

In [ ]:
txt = 'Does this chatbot work better than Alexa?'
r = predict(txt, clf)
pred = get_tag_from_prediction(r, tags, 0.20)
get_random_response_from_tag(pred, responses)

# Production

### Production class

In [ ]:
import logging
import numpy as np
import spacy
from sklearn.neural_network import MLPClassifier
from joblib import load

class Chatbot():
    
    path_classifier = 'utils/classifier.model'
    path_responses = 'utils/responses.data'
    path_tags = 'utils/tags.data'
    path_word_list = 'utils/word_list.data'
    
    classifier = None
    responses = None
    tags = None
    
     
    def __init__(self, threshold = 0.2, fallback = 'fallback', greeting = 'greeting'):
        
        self.nlp = spacy.load("en_core_web_sm")

        self.classifier = load(self.path_classifier)
        self.responses = load(self.path_responses)
        self.tags = load(self.path_tags)
        self.word_list = load(self.path_word_list)
        
        self.threshold = threshold #when below threshold chatbot returns a fallback answer because of the lacking confidence
        self.fallback = fallback #name of fallback tag
        self.greeting = greeting
    
    def lemmatizer(self, text):
        doc = self.nlp(text)
        return [d.lemma_ for d in doc]
        
    def get_tag_from_prediction(self, result, threshold):
        max_proba = np.max(result)
        if max_proba < threshold:
            return self.fallback
        predicted_tag = self.tags[np.argmax(result)]
        return predicted_tag
            
        
    def log_predictions(self, input_text, prediction_tag, probability, prediction_tag_if_not_fallback, response):
        logging.basicConfig(filename='logs/chatbot_predictions.log', encoding='utf-8')#, level=logging.DEBUG)
        s = f'{input_text}, {prediction_tag}, {probability}, {prediction_tag_if_not_fallback}, {response}'
        logging.debug(s)
    
    def bag_of_words(self, sentence):
        bog = np.zeros(len(self.word_list), dtype=np.float32)
        for idx, word in enumerate(self.word_list):
            if word in sentence:
                bog[idx] = 1
        return bog
    
    def pipe_new_input(self, text):
        if text == '': #otherwise error
            logging.info('Empty input')
            text = ' '
        text = self.lemmatizer(text)
        bog = self.bag_of_words(text)
        x = bog.reshape(1, bog.shape[0])
        return x
    
    def get_greeting(self):
        return self.get_random_response_from_tag(self.greeting)
    
    def get_random_response_from_tag(self, tag):
        
        return np.random.choice(self.responses[tag])
    
    def predict(self, input_text, log_status = True):
        x = self.pipe_new_input(input_text)
        
        result = self.classifier.predict_proba(x)
        max_proba = np.max(result)
        
        predicted_tag = self.get_tag_from_prediction(result, self.threshold)
        predicted_tag_except_fallback = self.get_tag_from_prediction(result, 0.0)
        
        response = self.get_random_response_from_tag(predicted_tag)
        
        if log_status:
            self.log_predictions(input_text,
                                    predicted_tag,
                                    max_proba,
                                    predicted_tag_except_fallback,
                                    response)
        return response

In [ ]:
cb = Chatbot(threshold = 0.25)

In [ ]:
cb.predict('Create a post for me', False)